In [ ]:
import pandas as pd
import numpy as np
import nrg_sql
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.metrics import mean_squared_log_error
def my_RMSLE(yhat,y):
    return np.sqrt(mean_squared_log_error(yhat,y))

In [3]:
df = pd.read_pickle('/data/nford/hackathon/df_train_test_all.pkl')

In [6]:
df.to_csv('sales.csv')

In [5]:
len(df)

3031446

In [4]:
df.head()

,index,date,family,id,onpromotion,sales,store_nbr,dcoilwtico,transactions,city,...,locale_name_l,year,month,day,dayofweek,paycheckday,type_n2,type_r2,type_l2,holiday_comb
0,0,2013-01-01,AUTOMOTIVE,0,0,0.0,1,93.14,NaN,Quito,...,NaN,2013,1,1,1,0.0,Holiday,NaN,NaN,Holiday
1,1,2013-01-01,BABY CARE,1,0,0.0,1,93.14,NaN,Quito,...,NaN,2013,1,1,1,0.0,Holiday,NaN,NaN,Holiday
2,2,2013-01-01,BEAUTY,2,0,0.0,1,93.14,NaN,Quito,...,NaN,2013,1,1,1,0.0,Holiday,NaN,NaN,Holiday
3,3,2013-01-01,BEVERAGES,3,0,0.0,1,93.14,NaN,Quito,...,NaN,2013,1,1,1,0.0,Holiday,NaN,NaN,Holiday
4,4,2013-01-01,BOOKS,4,0,0.0,1,93.14,NaN,Quito,...,NaN,2013,1,1,1,0.0,Holiday,NaN,NaN,Holiday


In [319]:
df = df.set_index('id')

In [320]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [321]:
df = df[df['date']>=pd.to_datetime('01-jan-2017')]

In [322]:
conditions = [
    df['date'] < pd.to_datetime('01-aug-2017'),
    df['date'].between(pd.to_datetime('01-aug-2017'), pd.to_datetime('15-aug-2017')),
    df['date'] > pd.to_datetime('15-aug-2017')
]
choices = ['train', 'validate', 'test']
df['data'] = np.select(conditions, choices, default='ERROR')

In [323]:
ref_date = pd.to_datetime('31-aug-2017')
df['back_counter'] = (ref_date - df.date).astype('timedelta64[D]')

In [324]:
df = df[['data', 'sales', 
         'store_nbr', 'family', 
         'year', 'month', 'day', 'dayofweek', 'paycheckday', 'holiday_comb', 
         'dcoilwtico', 'onpromotion', 'day_after_eq', 'back_counter']]

X = (df
    .pipe(pd.get_dummies, columns=['store_nbr', 'family', 'year', 'month', 'day', 'dayofweek', 'paycheckday', 'holiday_comb'], drop_first=True)
    .assign(intercept=1)  # Adds a column called 'intercept' with all values equal to 1.
)

In [325]:
X_train = X[X['data']=='train'].drop(columns='data')
#X_train = X_train.sample(frac=0.1, random_state=7)
y_train = X_train.pop("sales")

X_validate = X[X['data']=='validate'].drop(columns='data')
y_validate = X_validate.pop("sales")

X_test = X[X['data']=='test'].drop(columns='data')
X_test.pop('sales')

id
3000888   NaN
3000889   NaN
3000890   NaN
3000891   NaN
3000892   NaN
           ..
3029395   NaN
3029396   NaN
3029397   NaN
3029398   NaN
3029399   NaN
Name: sales, Length: 28512, dtype: float64

In [326]:
model = sm.GLM(
    y_train,
    X_train,
    family=sm.families.Poisson(),
)
result = model.fit()

In [327]:
ypred_train = result.predict(X_train)

In [328]:
ypred_validate = result.predict(X_validate)

In [329]:
my_RMSLE(ypred_validate, y_validate)

0.7318524798771754

In [330]:
X_train_validate = pd.concat([X_train, X_validate])
y_train_validate = pd.concat([y_train, y_validate])

In [331]:
model_pred = sm.GLM(
    y_train_validate,
    X_train_validate,
    family=sm.families.Poisson(),
)
result_pred = model_pred.fit()

In [332]:
ypred_test = result_pred.predict(X_test)

In [333]:
ypred_test = ypred_test.reset_index(drop=False)

In [334]:
ypred_test.columns=['id', 'sales']

In [335]:
ypred_test.to_csv('nate5.csv', index=False)

In [336]:
len(ypred_test)

28512